<a href="https://colab.research.google.com/github/ganonononon/kaggle_stock/blob/main/plotly%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Plotlyを使った株価可視化＆他の株価指標について解説

In [1]:
!pip install yfinance
!pip install plotly
!pip install yfinance==0.2.37

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.51
    Uninstalling yfinance-0.2.51:
      Successfully uninstalled yfinance-0.2.51


In [2]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import yfinance as yf

In [5]:
ticker = "GOOG"
period = "1y"
interval = "1d"
data_df  = yf.download(ticker, period=period, interval=interval)
data_df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-16,143.429993,145.839996,143.056000,144.080002,143.564163,19198900
2024-01-17,142.910004,143.410004,140.509995,142.889999,142.378418,17884500
2024-01-18,143.440002,145.585007,143.350006,144.990005,144.470901,18876800
2024-01-19,146.304993,148.039993,145.800003,147.970001,147.440231,27170900
2024-01-22,148.710007,150.014999,147.580002,147.710007,147.181168,21829200
...,...,...,...,...,...,...
2025-01-07,198.270004,202.139999,195.940002,196.710007,196.710007,16966800
2025-01-08,193.949997,197.639999,193.750000,195.389999,195.389999,14335300
2025-01-10,195.419998,197.619995,191.600006,193.169998,193.169998,20753800


In [6]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=data_df.index,
            open=data_df["Open"],
            high=data_df["High"],
            low=data_df["Low"],
            close=data_df["Close"],
        )
    ]
)

fig.update_layout(
    title="Google 株価",
    xaxis_title="日付",
    yaxis_title="株価 (USD)"
)

fig.show()

複数のグラフを表示する

In [11]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.2, subplot_titles=("ローソク足チャート", "出来高"))

fig.add_trace(go.Candlestick(
            x=data_df.index,
            open=data_df["Open"],
            high=data_df["High"],
            low=data_df["Low"],
            close=data_df["Close"],
            name="株価"
        ), row=1, col=1
)

fig.add_trace(go.Bar(
        x=data_df.index,
        y=data_df["Volume"],
        marker_color="blue",
        name="出来高"
    ) , row=2, col=1)


fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis2_rangeslider_visible=True,
    xaxis2_title="日付",
    yaxis_title="株価",
    yaxis2_title="出来高",
)

fig.show()


# 株価のテクニカル指標

MACDライン：短期EMA(指数平滑移動平均線)と長期EMAの差。一般的には、12日EMAと26日EMAの差を用いる。

シグナルライン：MACSラインのEMA。通常、９日間のMACDラインのEMAを使用します。

ヒストグラム：MACDラインとシグナルラインの差。MACDヒストグラムは市場の勢いを示し、MACDラインがシグナルラインを上回る時に生の値(強気)、下回る時に負の値(弱気)をとる。

In [17]:
short_ema = data_df["Close"].ewm(span=12, adjust=False).mean()
long_ema = data_df["Close"].ewm(span=26, adjust=False).mean()

macd = short_ema - long_ema

signal = macd.ewm(span=9, adjust=False).mean()


hist = macd - signal

data_df["MACD"] = macd
data_df["Signal"] = signal
data_df["Histgram"] = hist
data_df

,Open,High,Low,Close,Adj Close,Volume,MACD,Signal,Histgram
Date,,,,,,,,,
2024-01-16,143.429993,145.839996,143.056000,144.080002,143.564163,19198900,0.000000,0.000000,0.000000
2024-01-17,142.910004,143.410004,140.509995,142.889999,142.378418,17884500,-0.094929,-0.018986,-0.075943
2024-01-18,143.440002,145.585007,143.350006,144.990005,144.470901,18876800,-0.000700,-0.015329,0.014629
2024-01-19,146.304993,148.039993,145.800003,147.970001,147.440231,27170900,0.310855,0.049908,0.260947
2024-01-22,148.710007,150.014999,147.580002,147.710007,147.181168,21829200,0.530668,0.146060,0.384608
...,...,...,...,...,...,...,...,...,...
2025-01-07,198.270004,202.139999,195.940002,196.710007,196.710007,16966800,4.217713,4.605052,-0.387340
2025-01-08,193.949997,197.639999,193.750000,195.389999,195.389999,14335300,4.053963,4.494834,-0.440871
2025-01-10,195.419998,197.619995,191.600006,193.169998,193.169998,20753800,3.702377,4.336343,-0.633966


In [19]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.2, subplot_titles=("ローソク足チャート", "MACD"))

fig.add_trace(go.Candlestick(
            x=data_df.index,
            open=data_df["Open"],
            high=data_df["High"],
            low=data_df["Low"],
            close=data_df["Close"],
            name="株価"
        ), row=1, col=1
)

fig.add_trace(go.Scatter(x=data_df.index, y=data_df["MACD"], name="MACD", mode="lines"), row=2, col=1)
fig.add_trace(go.Scatter(x=data_df.index, y=data_df["Signal"], name="Signal", mode="lines"), row=2, col=1)
fig.add_trace(go.Bar(x=data_df.index, y=data_df["Histgram"], name="Histgram"), row=2, col=1)


fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis2_rangeslider_visible=True,
    xaxis2_title="日付",
    yaxis_title="株価",
    yaxis2_title="出来高",
)

fig.show()